In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
!nvidia-smi

Mon Jan  1 22:09:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:22:00.0 Off |                  Off |
| 30%   46C    P8              27W / 300W |  38222MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
import numpy as np
import torch
import os
import re
import json
import argparse
import random
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from model import T5ForMultimodalGeneration

In [3]:
img_dim = 1408
model = T5ForMultimodalGeneration.from_pretrained("/home/jiayipan/code/ICML_GUI/Auto-UI/weights/Auto-UI-Base", img_dim) 

In [4]:
from infer_utils import ImageFeatureExtractor
img_extractor = ImageFeatureExtractor()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from PIL import Image
img = Image.open("/home/jiayipan/code/ICML_GUI/example_ios.jpeg")
img_feat = img_extractor.to_feat(img)

In [6]:
model_path = "/home/jiayipan/code/ICML_GUI/Auto-UI/weights/Auto-UI-Base"
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [7]:
inputs = tokenizer("Open Chrome", return_tensors="pt")

In [8]:
model.cuda()

T5ForMultimodalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo): 

In [12]:
img_feat = img_feat.float().cuda()

In [14]:
out = model.generate(
    input_ids = inputs["input_ids"].to("cuda"),
    attention_mask = inputs["attention_mask"].to("cuda"),
    image_ids = img_feat.unsqueeze(0).to("cuda"),
)

/home/jiayipan/miniconda3/envs/GPML/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [15]:
tokenizer.decode(out[0])

'<pad> Action Plan: [DUAL_POINT,STATUS_TASK_COMPLE'

In [ ]:
training_data = load_data(args, "train")